# 在TinyMind上运行代码

# 导入工具包

In [5]:
# import sys
# tensorflow 工具包
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
# 打印运行时间
import time
# 导入tinymind环境自带的变量
try: 
    from tinyenv.flags import flags
except ImportError:
    # 若在本地运行，则自动生成相同的class
    class flags(object):
        def __init__(self):
            self.iterations=20000
            self.learning_rate = 0.005
            self.batch_size = 64
            self.dropout = 0.5
            self.decay = 0.95
            self.output_dir = './output'
            self.data_dir = './tmp/mnist'
            self.f1_ck=[6,6]
            self.f1_cc=32
            self.f2_ck=[6,6]
            self.f2_cc=64
            self.features=1024
            self.l_r_s=4e-3
            self.l_r_e = 1e-4            
            self.l2_regularizer=7e-5
#实例化class
FLAGS = flags()

print('import is over')


import is over


# 定义主要运行函数

In [6]:

class Mnist_CNN(object):
    """mnist的CNN实现，默认参数test中正确率为99%左右
    
    第一层: 卷积核为f1_ck=[6,6], 1通道->f1_cc=32通道; (1152)
    最大池化, 核为f1_pc=[2,2], 步长为f1_ps=2
    第二层: 卷积核为f2_ck=[6,6], 32通道->f2_cc=64通道; (2304)
    最大池化, 核为f2_pk=[2,2], 步长为f2_ps=[2,2]
    先平均池化, 核为全部数据; 
    再卷积，核为1, 输出通道为features=1024的向量 (1024)
    dropout概率为drop=0.5之后, 卷积为10的向量;(10) 经softmax激活
    损失函数为：交叉熵+l2_regularizer=7e-5*L2损失
    利用Adam方法, 初始学习率为l_r_s=4e-3, 循环for_=6000次
    最终训练结束时学习率为l_r_e=1e-4， 每次batch=64,
    未知数共(4490)个
    """

    def __init__(self,f1_ck=[6,6],f1_cc=32,f1_pc=[2,2],f1_ps=2,
                 f2_ck=[6,6],f2_cc=64,f2_pk=[2,2],f2_ps=2,
                 features=1024, drop=0.5, l2_regularizer=7e-5,
                 l_r_s=4e-3, l_r_e = 1e-4, for_=20000,batch=64):
        #学习率衰减
        l_r_decay = (l_r_s - l_r_e)/for_
        lr=l_r_s
        # 导入数据
        mnist = input_data.read_data_sets(FLAGS.data_dir+'/mnist',  one_hot=True,)
        
        #创建新的空白运行图
        tf.reset_default_graph()
        sess = tf.InteractiveSession()
        
        # 输入数据维度
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784])
            y_ = tf.placeholder(tf.float32, [None, 10])
        
        # 将x转为4阶张量
        with tf.name_scope('reshape'):
            x_image = tf.reshape(x, [-1, 28, 28, 1])
        
        # 第一层之卷积,卷积核为[6,6]
        # [-1, 28, 28, 1] => [-1, 28, 28, 32]
        with tf.name_scope('conv1'):
            h_conv1 = tf.contrib.slim.conv2d(x_image, f1_cc, f1_ck, 
                                             padding='SAME', 
                                             activation_fn=tf.nn.tanh)
        #tf.summary.scalar('conv1_l2_regularizer', tf.contrib.layers.l2_regularizer(0.1)(h_conv1))
        
        # 第一层之最大池化, 核[2,2], 步长2
        # [-1, 28, 28, 32] => [-1, 14, 14, 32]
        with tf.name_scope('pool1'):
            h_pool1 = tf.contrib.slim.max_pool2d(h_conv1, f1_pc, stride=f1_ps, 
                                                 padding='VALID')  
        # 第二层之卷积,卷积核为[6,6]
        # [-1, 14, 14, 32] => [-1, 14, 14, 64]
        with tf.name_scope('conv2'):
            h_conv2 = tf.contrib.slim.conv2d(h_pool1, f2_cc, f2_ck, 
                                             padding='SAME', 
                                             activation_fn=tf.nn.relu)
        
        # 第二层之最大池化, 核[2,2], 步长2
        # [-1, 14, 14, 64] => [-1, 7, 7, 64]
        with tf.name_scope('pool2'):
            h_pool2 = tf.contrib.slim.max_pool2d(h_conv2, f2_pk, stride=f2_ps, 
                                                 padding='VALID')
        
        
        # 第三层之先池化再卷积,
        # 池化核[7,7]，步长1，[-1, 7, 7, 64] => [-1, 1, 1, 64]
        # 卷积核[1,1]，[-1, 1, 1, 64] => [-1, 1, 1, 1024]
        with tf.name_scope('fc1'):
            h_pool2_flat = tf.contrib.slim.avg_pool2d(h_pool2, 
                                                      h_pool2.shape[1:3], 
                                                      stride=[1, 1], 
                                                      padding='VALID')
            h_fc1 = tf.contrib.slim.conv2d(h_pool2_flat, features, [1, 1], 
                                           activation_fn=tf.nn.relu)
        
        # 避免过拟合，加入Dropout
        with tf.name_scope('fc1_dropout'):
            keep_prob = tf.placeholder(tf.float32)
            h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        tf.summary.histogram('fc1_dropout', h_fc1_drop)
        tf.summary.scalar('fc1_dropout_l2_regularizer', tf.contrib.layers.l2_regularizer(0.1)(h_fc1_drop))
        
        # 卷积核[1,1]，[-1, 1, 1, 1024] => [-1, 1, 1, 10]
        with tf.name_scope('fc2'):
            y = tf.squeeze(tf.contrib.slim.conv2d(h_fc1_drop, 10, [1,1], 
                                                  activation_fn=None))
        # 定义损失函数
        with tf.name_scope('loss'):
            # 交叉熵损失
            cross_entropy = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
            # 所有VARIABLES的l2损失
            l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
            total_loss = cross_entropy + l2_regularizer*l2_loss
        tf.summary.scalar('cross_entropy', cross_entropy)
        tf.summary.scalar('l2_loss', l2_loss)
        tf.summary.scalar('total_loss', total_loss)
        
        # 定义评价函数，正确率
        with tf.name_scope('accuracy'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
            
        # 优化方法
        with tf.name_scope('train_step'):
            #global_step = tf.Variable(0)  
            #learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase=True)  
            learning_rate = tf.placeholder(tf.float32)
            #train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
            #train_step = tf.train.MomentumOptimizer(learning_rate).minimize(total_loss)
        tf.summary.scalar('learning_rate', learning_rate)
        
        # 全部变量初始化
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        
        # 构建summary
        summary_op = tf.summary.merge_all()
        train_summary_writer = tf.summary.FileWriter(FLAGS.output_dir+'/train/', 
                                                     graph = sess.graph,)
                                                     #filename_suffix='train_{}'.format(***))
        test_summary_writer = tf.summary.FileWriter(FLAGS.output_dir+'/test/', )
                                                    #filename_suffix='test_{}'.format(***))
        
        print('Session is over')
        
        # Train
        start = time.time()
        for step in range(for_):
            batch_xs, batch_ys = mnist.train.next_batch(batch)
            lr -= l_r_decay
            _, total_loss_value = sess.run([train_step, total_loss], 
                                           feed_dict={x: batch_xs, 
                                                      y_: batch_ys, 
                                                      learning_rate:lr, 
                                                      keep_prob:drop})


            if (step+1)%100==0:
                summary_test,acc = sess.run([summary_op,accuracy],
                                           feed_dict={x: mnist.test.images,
                                                      y_: mnist.test.labels,
                                                      learning_rate:lr,
                                                      keep_prob:1.0})
                test_summary_writer.add_summary(summary_test, step+1)
                
                summary_train,acc_train = sess.run([summary_op,accuracy],
                                           feed_dict={x: batch_xs, 
                                          y_: batch_ys, 
                                          learning_rate:lr, 
                                          keep_prob:drop})
                train_summary_writer.add_summary(summary_train, step+1)
            if (step+1)%500==0:
                print('accuracy of test at step {0}:{1}, time is {2}'.format(step+1, 
                                                    acc, int(time.time()-start)))

        sess.close()        
        print('teain is over, totle time', int(time.time()-start))

print('class is over')



class is over


# 实际运行

In [ ]:
mni = Mnist_CNN(f1_ck=FLAGS.f1_ck, f1_cc=FLAGS.f1_cc, f2_ck=FLAGS.f2_ck, 
                f2_cc=FLAGS.f2_cc, features=FLAGS.features, drop=FLAGS.dropout, 
                l2_regularizer= FLAGS.l2_regularizer, l_r_s=FLAGS.l_r_s, l_r_e = FLAGS.l_r_e, 
                for_=FLAGS.iterations, batch=FLAGS.batch_size)


# 结果分析